In [1]:
import polars as pl 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
!pip install pandas
import pandas as pd

In [3]:
!pip install lets-plot

In [4]:
from lets_plot import *
LetsPlot.setup_html()

In [7]:
grocery = pl.read_csv("GroceryDataset.csv")

In [8]:
grocery.head()

Sub Category,Price,Discount,Rating,Title,Currency,Feature,Product Description
str,str,str,str,str,str,str,str
"""Bakery & Desserts""","""$56.99 ""","""No Discount""","""Rated 4.3 out of 5 stars based…","""David’s Cookies Mile High Pean…","""$""","""""10"""" Peanut Butter Cake Certi…","""A cake the dessert epicure wil…"
"""Bakery & Desserts""","""$159.99 ""","""No Discount""","""Rated 5 out of 5 stars based o…","""The Cake Bake Shop 8"" Round Ca…","""$""","""Spiced Carrot Cake with Cream …","""Due to the perishable nature o…"
"""Bakery & Desserts""","""$44.99 ""","""No Discount""","""Rated 4.1 out of 5 stars based…","""St Michel Madeleine, Classic F…","""$""","""100 count Individually wrapped…","""Moist and buttery sponge cakes…"
"""Bakery & Desserts""","""$39.99 ""","""No Discount""","""Rated 4.7 out of 5 stars based…","""David's Cookies Butter Pecan M…","""$""","""Butter Pecan Meltaways 32 oz 2…","""These delectable butter pecan …"
"""Bakery & Desserts""","""$59.99 ""","""No Discount""","""Rated 4.5 out of 5 stars based…","""David’s Cookies Premier Chocol…","""$""","""""10"" Four Layer Chocolate Cake…","""A cake the dessert epicure wil…"


In [9]:
# makes Price a float instead of string

grocery = grocery.with_columns(
    pl.col("Price")
      .str.replace_all(r'[$,\s"]', "")
      .cast(pl.Float64, strict=False)
      .alias("Price")
)

In [10]:
grocery["Discount"].unique()

Discount
str
"""After $9.30 OFF"""
"""After $2.40 OFF"""
"""After $3.10 OFF"""
"""Limit 5 Per Member"""
"""After $2.80 OFF"""
…
"""After $3.30 OFF"""
"""After $2.70 OFF"""
"""After $20 OFF"""


In [11]:
grocery = grocery.with_columns(
    pl.col("Rating")
    .str.extract(r"(\d+\.\d+)")
    .cast(pl.Float64)
)

In [12]:
grocery

Sub Category,Price,Discount,Rating,Title,Currency,Feature,Product Description
str,f64,str,f64,str,str,str,str
"""Bakery & Desserts""",56.99,"""No Discount""",4.3,"""David’s Cookies Mile High Pean…","""$""","""""10"""" Peanut Butter Cake Certi…","""A cake the dessert epicure wil…"
"""Bakery & Desserts""",159.99,"""No Discount""",null,"""The Cake Bake Shop 8"" Round Ca…","""$""","""Spiced Carrot Cake with Cream …","""Due to the perishable nature o…"
"""Bakery & Desserts""",44.99,"""No Discount""",4.1,"""St Michel Madeleine, Classic F…","""$""","""100 count Individually wrapped…","""Moist and buttery sponge cakes…"
"""Bakery & Desserts""",39.99,"""No Discount""",4.7,"""David's Cookies Butter Pecan M…","""$""","""Butter Pecan Meltaways 32 oz 2…","""These delectable butter pecan …"
"""Bakery & Desserts""",59.99,"""No Discount""",4.5,"""David’s Cookies Premier Chocol…","""$""","""""10"" Four Layer Chocolate Cake…","""A cake the dessert epicure wil…"
…,…,…,…,…,…,…,…
"""Snacks""",23.99,"""No Discount""",null,"""Oberto Thin Style Smoked Sausa…","""$""","""Cocktail Pepperoni Smoked Sa…","""Cocktail PepperoniSmoked Sausa…"
"""Snacks""",49.99,"""No Discount""",null,"""Cheetos Crunchy, Original, 2.1…","""$""","""Made with Real Cheese""","""64-count2.1 oz Bags"""
"""Snacks""",22.99,"""No Discount""",null,"""Sabritas Chile & Limon Mix, Va…","""$""","""Chile & Limón Mix Variety Pa…","""8-Doritos Dinamita Chile Limón…"


In [13]:
grocery = grocery.with_columns(
    pl.when(pl.col("Discount").str.strip_chars() == "No Discount")
      .then(pl.lit(0.0))
      .otherwise(
          pl.col("Discount")
            .str.extract(r'(\d+(?:\.\d+)?)')
      )
      .cast(pl.Float64, strict=False)
      .alias("Discount")
)


In [14]:
grocery

Sub Category,Price,Discount,Rating,Title,Currency,Feature,Product Description
str,f64,f64,f64,str,str,str,str
"""Bakery & Desserts""",56.99,0.0,4.3,"""David’s Cookies Mile High Pean…","""$""","""""10"""" Peanut Butter Cake Certi…","""A cake the dessert epicure wil…"
"""Bakery & Desserts""",159.99,0.0,null,"""The Cake Bake Shop 8"" Round Ca…","""$""","""Spiced Carrot Cake with Cream …","""Due to the perishable nature o…"
"""Bakery & Desserts""",44.99,0.0,4.1,"""St Michel Madeleine, Classic F…","""$""","""100 count Individually wrapped…","""Moist and buttery sponge cakes…"
"""Bakery & Desserts""",39.99,0.0,4.7,"""David's Cookies Butter Pecan M…","""$""","""Butter Pecan Meltaways 32 oz 2…","""These delectable butter pecan …"
"""Bakery & Desserts""",59.99,0.0,4.5,"""David’s Cookies Premier Chocol…","""$""","""""10"" Four Layer Chocolate Cake…","""A cake the dessert epicure wil…"
…,…,…,…,…,…,…,…
"""Snacks""",23.99,0.0,null,"""Oberto Thin Style Smoked Sausa…","""$""","""Cocktail Pepperoni Smoked Sa…","""Cocktail PepperoniSmoked Sausa…"
"""Snacks""",49.99,0.0,null,"""Cheetos Crunchy, Original, 2.1…","""$""","""Made with Real Cheese""","""64-count2.1 oz Bags"""
"""Snacks""",22.99,0.0,null,"""Sabritas Chile & Limon Mix, Va…","""$""","""Chile & Limón Mix Variety Pa…","""8-Doritos Dinamita Chile Limón…"


In [15]:
grocery = grocery.drop_nulls(["Price", "Discount", "Rating", "Sub Category"])

In [16]:
display(grocery.head())
print(grocery.select([
    pl.col("Price").mean().alias("avg_price"),
    pl.col("Discount").mean().alias("avg_disc"),
    pl.col("Rating").mean().alias("avg_rating"),
    pl.len().alias("n_rows")
]))

Sub Category,Price,Discount,Rating,Title,Currency,Feature,Product Description
str,f64,f64,f64,str,str,str,str
"""Bakery & Desserts""",56.99,0.0,4.3,"""David’s Cookies Mile High Pean…","""$""","""""10"""" Peanut Butter Cake Certi…","""A cake the dessert epicure wil…"
"""Bakery & Desserts""",44.99,0.0,4.1,"""St Michel Madeleine, Classic F…","""$""","""100 count Individually wrapped…","""Moist and buttery sponge cakes…"
"""Bakery & Desserts""",39.99,0.0,4.7,"""David's Cookies Butter Pecan M…","""$""","""Butter Pecan Meltaways 32 oz 2…","""These delectable butter pecan …"
"""Bakery & Desserts""",59.99,0.0,4.5,"""David’s Cookies Premier Chocol…","""$""","""""10"" Four Layer Chocolate Cake…","""A cake the dessert epicure wil…"
"""Bakery & Desserts""",59.99,0.0,4.4,"""David's Cookies Mango & Strawb…","""$""","""2-count Strawberry Cheesecake …","""Strawberry Cheesecake: There's…"


shape: (1, 4)
┌────────────┬──────────┬────────────┬────────┐
│ avg_price  ┆ avg_disc ┆ avg_rating ┆ n_rows │
│ ---        ┆ ---      ┆ ---        ┆ ---    │
│ f64        ┆ f64      ┆ f64        ┆ u32    │
╞════════════╪══════════╪════════════╪════════╡
│ 121.871166 ┆ 1.403825 ┆ 4.370856   ┆ 549    │
└────────────┴──────────┴────────────┴────────┘


In [17]:
per_cat = grocery.group_by("Sub Category").agg(
    pl.col("Price").mean().alias("cat_price_mean"),
    pl.col("Price").std(ddof=1).alias("cat_price_sd"),
    pl.len().alias("cat_count")
)

g2 = grocery.join(per_cat, on="Sub Category", how="left").with_columns(
    pl.col("Price").log().alias("log_price"),
    (pl.col("Discount") / 100).alias("disc_frac"),
    (pl.col("Price") * (1 - (pl.col("Discount")/100))).alias("price_after_disc"),
    ( (pl.col("Price") - pl.col("cat_price_mean")) / pl.col("cat_price_sd") ).alias("price_z"),
    pl.col("cat_count"),
    (
      pl.col("Title").fill_null("") + " " +
      pl.col("Feature").fill_null("") + " " +
      pl.col("Product Description").fill_null("")
    ).str.strip_chars().alias("text_all")
).drop_nulls(["price_z"])

target = "Rating"
num_cols = ["Price","Discount","log_price","disc_frac","price_after_disc","price_z","cat_count"]
cat_cols = ["Sub Category"]
text_col = "text_all"
all_cols = [*num_cols, *cat_cols, text_col, target]

dfm = g2.select(all_cols)
dfm.shape


(549, 10)

In [18]:
dfm.head()

Price,Discount,log_price,disc_frac,price_after_disc,price_z,cat_count,Sub Category,text_all,Rating
f64,f64,f64,f64,f64,f64,u32,str,str,f64
56.99,0.0,4.042876,0.0,56.99,-0.289053,22,"""Bakery & Desserts""","""David’s Cookies Mile High Pean…",4.3
44.99,0.0,3.80644,0.0,44.99,-0.659489,22,"""Bakery & Desserts""","""St Michel Madeleine, Classic F…",4.1
39.99,0.0,3.688629,0.0,39.99,-0.813838,22,"""Bakery & Desserts""","""David's Cookies Butter Pecan M…",4.7
59.99,0.0,4.094178,0.0,59.99,-0.196444,22,"""Bakery & Desserts""","""David’s Cookies Premier Chocol…",4.5
59.99,0.0,4.094178,0.0,59.99,-0.196444,22,"""Bakery & Desserts""","""David's Cookies Mango & Strawb…",4.4


In [19]:
grocery = grocery.with_columns(
    (pl.col("Discount")/100).alias("disc_frac"),
    (pl.col("Price") * (1 - pl.col("Discount")/100)).alias("price_after_disc")
)

In [20]:
filt = (
    (pl.col("Currency") == "$") &
    (pl.col("Price").is_between(5, 5000)) &
    pl.col("Rating").is_not_null()
)
g = grocery.filter(filt)
g.head()

Sub Category,Price,Discount,Rating,Title,Currency,Feature,Product Description,disc_frac,price_after_disc
str,f64,f64,f64,str,str,str,str,f64,f64
"""Bakery & Desserts""",56.99,0.0,4.3,"""David’s Cookies Mile High Pean…","""$""","""""10"""" Peanut Butter Cake Certi…","""A cake the dessert epicure wil…",0.0,56.99
"""Bakery & Desserts""",44.99,0.0,4.1,"""St Michel Madeleine, Classic F…","""$""","""100 count Individually wrapped…","""Moist and buttery sponge cakes…",0.0,44.99
"""Bakery & Desserts""",39.99,0.0,4.7,"""David's Cookies Butter Pecan M…","""$""","""Butter Pecan Meltaways 32 oz 2…","""These delectable butter pecan …",0.0,39.99
"""Bakery & Desserts""",59.99,0.0,4.5,"""David’s Cookies Premier Chocol…","""$""","""""10"" Four Layer Chocolate Cake…","""A cake the dessert epicure wil…",0.0,59.99
"""Bakery & Desserts""",59.99,0.0,4.4,"""David's Cookies Mango & Strawb…","""$""","""2-count Strawberry Cheesecake …","""Strawberry Cheesecake: There's…",0.0,59.99


Which categories have the highest average rating? (only showing categories with >20 items)

In [21]:
q1 = (
    g.group_by("Sub Category")
     .agg(
         pl.len().alias("n"),
         pl.col("Rating").mean().alias("avg_rating"),
         pl.col("Price").mean().alias("avg_price"),
         pl.col("Discount").mean().alias("avg_discount_pct")
     )
     .filter(pl.col("n") >= 20)
     .sort("avg_rating", descending=True)
)
q1

Sub Category,n,avg_rating,avg_price,avg_discount_pct
str,u32,f64,f64,f64
"""Coffee""",36,4.530556,42.878889,0.277778
"""Pantry & Dry Goods""",42,4.480952,68.894762,0.952381
"""Gift Baskets""",67,4.426866,112.542239,2.238806
"""Cleaning Supplies""",37,4.418919,46.61973,0.408108
"""Snacks""",33,4.369697,45.505152,0.606061
"""Meat & Seafood""",113,4.293805,241.05177,2.300885
"""Seafood""",37,4.275676,196.611081,1.891892
"""Floral""",45,4.248889,68.145556,0.0
"""Bakery & Desserts""",22,4.163636,66.353636,0.0


In [ ]:
!pip install pyarrow

In [ ]:
q1_sorted = q1.sort("avg_rating")

data_q1 = {
    "Sub Category": q1_sorted["Sub Category"].to_list(),
    "avg_rating":   q1_sorted["avg_rating"].to_list()
}

ggplot(data_q1, aes(x="avg_rating", y="Sub Category")) + \
    geom_bar(stat="identity", orientation="y") + \
    labs(title="Average Rating by Sub-Category (n ≥ 20)",
         x="Average Rating", y="Sub-Category")

In [23]:
# ggplot(q1, aes(x="Sub Category", y="avg_rating")) + \
# geom_col(fill="skyblue")

ggplot(q1, aes(x="Sub Category", y="avg_rating"))

As seen in this, Coffee has the highest average rating of all the subcategories with more than 20 items.

In [ ]:
g = grocery.filter(
    (pl.col("Currency") == "$") &
    pl.col("Price").is_between(5, 5000) &
    pl.col("Rating").is_not_null()
)

q_discounts = (
    g.group_by("Sub Category")
     .agg(
        pl.len().alias("n"),
        pl.col("Discount").mean().alias("avg_discount_pct"),
        pl.col("Price").mean().alias("avg_price")
     )
     .filter(pl.col("n") >= 20)
     .sort("avg_discount_pct", descending=True)
)
q_discounts


Sub Category,n,avg_discount_pct,avg_price
str,u32,f64,f64
"""Meat & Seafood""",113,2.300885,241.05177
"""Gift Baskets""",67,2.238806,112.542239
"""Seafood""",37,1.891892,196.611081
"""Pantry & Dry Goods""",42,0.952381,68.894762
"""Snacks""",33,0.606061,45.505152
"""Cleaning Supplies""",37,0.408108,46.61973
"""Coffee""",36,0.277778,42.878889
"""Floral""",45,0.0,68.145556
"""Bakery & Desserts""",22,0.0,66.353636


In [25]:
ggplot(q_discounts, aes(x="avg_discount_pct", y="Sub Category")) + \
    geom_bar(stat="identity", orientation="y") + \
    labs(title="Highest Average Discount Percentage by Sub Category",
         x="Discount Percentage", y="Sub-Category")

In [26]:
!pip install nbconvert pyppeteer

  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fastjsonschema-2.21.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.27.1-cp311-cp311-win_amd64.whl.metadata (4.3 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached fastjsonschema-2.21.2-py3-none-any.whl (24 kB)
Using cached jsonschema-4.25.1-py3-none-a